In [1]:
import warnings as wns
wns.filterwarnings('ignore')

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


입력 데이터를 판다스 데이터프레임으로 저장한다.  
입력 데이터는 음식 관련 및 스포츠 관련 지문으로 구성되어 있다.

In [15]:
paragraph_dict_list = [
     {'paragraph': 'Dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
     {'paragraph': 'Srvice can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
     {'paragraph': 'Portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
     {'paragraph': 'We started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
     {'paragraph': 'The biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
     {'paragraph': 'The garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
     {'paragraph': 'Our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
     {'paragraph': 'What i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
     {'paragraph': 'The drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
     {'paragraph': 'Despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},

     {'paragraph': 'The four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
     {'paragraph': 'The briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
     {'paragraph': 'Stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
     {'paragraph': 'When it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
     {'paragraph': 'The team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
     {'paragraph': 'The perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
     {'paragraph': 'Liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
     {'paragraph': 'Alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
     {'paragraph': 'But the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
     {'paragraph': 'Then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the third set there were exhilarating rallies with both chasing to the net both retrieving what looked like winning shots nadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]
df = pd.DataFrame(paragraph_dict_list)
# df = df[['paragraph', 'category']]

In [4]:
df.head()

,paragraph,category
0,Dishplace is located in sunnyvale downtown the...,food
1,Srvice can be slower during busy hours but our...,food
2,Portions are huge both french toast and their ...,food
3,We started with apps going the chicken and waf...,food
4,The biscuits and gravy was too salty two peopl...,food


In [5]:
df.tail()

,paragraph,category
15,The perseyside outfit finished in fourth place...,sports
16,Liverpool fc will return to premier league act...,sports
17,Alisson signed for liverpool fc from as roma t...,sports
18,But the rankings during that run-in to new yor...,sports
19,Then came the oh-so-familiar djokovic-nadal no...,sports


데이터 전처리   
LSTM 모델이 입력 데이터를 처리할 수 있도록 수치값으로 변경한다.  
텍스트인 입력값을 수치로 변경하기 위해 지문에 사용된 모든 단어들을 모아서 중복을 제거한 후 단어 리스트를 만든다.

In [6]:
# 중복을 제거해서 지문에 사용된 단어를 모으기 위해 빈 set을 선언한다.
results = set()
# 판다스의 Series 객체 => 1차원 배열과 같은 자료구조로 index가 같이 출력된다.

# print(type(df))                  <class 'pandas.core.frame.DataFrame'>
# print(type(df['paragraph']))     <class 'pandas.core.series.Series'>
# print(type(df['paragraph'].str)) <class 'pandas.core.strings.accessor.StringMethods'>
# print(type(df['paragraph'].str.lower()))             <class 'pandas.core.series.Series'>
# print(type(df['paragraph'].str.lower().str))         <class 'pandas.core.strings.accessor.StringMethods'>
# print(type(df['paragraph'].str.lower().str.split())) <class 'pandas.core.series.Series'>
# print(type(df['paragraph'].str.lower().str.split().apply(results.update))) <class 'pandas.core.series.Series'>

# 데이터프레임의 'paragraph' 열 (Series 객체)를 모두 소문자로 변환한 후 공백을 경계로 나눠서 set에 저장한다.
df['paragraph'].str.lower().str.split().apply(results.update)
print(len(results))
print(results)

537
{'breakthroughs', 'no1', 'miami', 'stunning', 'madrid', 'be', 'italico', 'roma', 'certainly', 'separate', 'did', 'williams', 'years', '10-9', 'portion', 'avoided', 'minutes', 'semis', 'world', 'hours', 'american', '(and', 'via', 'does', 'if', 'new', 'this', 'competition', 'pulled', 'between', 'two-sets', 'down', 'area', 'most', 'when', 'tie-break', 'within', 'hope', 'small', 'cup', 'ostapenko', 'crumbs', 'succeed', 'were', 'action', 'fourth', 'form', 'likely', 'hodgson', 'no-quarter-given', 'say', 'prepare', 'special)', 'smash', 'seeded', 'helped', 'fellow', 'games', 'year', 'long', 'us', 'courteous', 'our', 'slow', 'iced', 'names', 'gravy', 'business', 'we', 'as', 'know', 'bookmarked', 'slides', 'ai', 'bit', 'back', 'simona', 'in', 'missed', 'to', 'more', 'showed', 'tournaments', 'saturday', 'flavors', 'djokovic', 'various', 'here', 'wasnt', 'leicester', 'reaching', 'dominance', 'bottom', 'came', 'results', 'big', 'ranks', 'quartet', 'serving', 'thought', 'semi-final', 'briton', '

In [7]:
# set에 저장된 데이터를 딕셔너리로 변환한다.
# {0: 'succeed', 1: 'eat', 2: 'dragged', ..., 534: 'eliminated', 535: 'where', 536: 'performances'}
# enumerate() : 인수로 지정된 객체의 인덱스와 데이터를 리턴한다.
# for index, word in enumerate(results):
    # print(index, word)
    
idx2word = dict(enumerate(results))

# print(type(idx2word))
# print(idx2word)
# {index: word, ...} 형태의 딕셔너리를 {word: index} 형태의 딕셔너리로 변환한다.
# {'succeed': 0, 'eat': 1, 'dragged': 2, ..., 'eliminated': 534, 'where': 535, 'performances': 536}

word2idx = {value: key for key, value in idx2word.items()}

# print(type(word2idx))
# print(word2idx)

In [8]:
print(word2idx['one'], word2idx['week']) 
print(word2idx['bread'])
print(word2idx['tournament'])

390 289
480
291


word2idx 딕셔너리를 활용해서 모든 지문을 수치로 전환한다.

In [9]:
# 지문을 구성하는 모든 단어를 수치로 전환하는 함수를 만든다.
def encode_paragraph(paragraph):
    words = paragraph.split(' ')
    encoded = []
    for word in words:
        encoded.append([word2idx[word.lower()]])
    return encoded


# 지문이 수치로 변환된 결과를 데이터프레임에 추가한다.
df['enc_paragraph'] = df.paragraph.apply(encode_paragraph)
df.head()

,paragraph,category,enc_paragraph
0,Dishplace is located in sunnyvale downtown the...,food,"[[240], [111], [294], [77], [435], [253], [530..."
1,Srvice can be slower during busy hours but our...,food,"[[346], [132], [5], [314], [135], [262], [19],..."
2,Portions are huge both french toast and their ...,food,"[[282], [208], [478], [522], [334], [235], [42..."
3,We started with apps going the chicken and waf...,food,"[[68], [266], [257], [364], [428], [153], [128..."
4,The biscuits and gravy was too salty two peopl...,food,"[[153], [470], [427], [66], [439], [401], [507..."


분류 항목(food[1, 0], sports[0, 1])을 one hot encoding으로 수치로 변경한다.

In [10]:
# 분류 항목을 one hot encoding으로 변환하는 함수를 만든다.
def encode_category(category):
    if category == 'food':
        return [1, 0]
    else:
        return [0, 1]

# 분류 항목이 one hot encoding으로 변환된 결과를 데이터프레임에 추가한다.
df['enc_category'] = df.category.apply(encode_category) # target, label, class
df

,paragraph,category,enc_paragraph,enc_category
0,Dishplace is located in sunnyvale downtown the...,food,"[[240], [111], [294], [77], [435], [253], [530...","[1, 0]"
1,Srvice can be slower during busy hours but our...,food,"[[346], [132], [5], [314], [135], [262], [19],...","[1, 0]"
2,Portions are huge both french toast and their ...,food,"[[282], [208], [478], [522], [334], [235], [42...","[1, 0]"
3,We started with apps going the chicken and waf...,food,"[[68], [266], [257], [364], [428], [153], [128...","[1, 0]"
4,The biscuits and gravy was too salty two peopl...,food,"[[153], [470], [427], [66], [439], [401], [507...","[1, 0]"
5,The garlic fries were a great starter (and a h...,food,"[[153], [180], [144], [43], [157], [115], [461...","[1, 0]"
6,Our meal was excellent i had the pasta ai form...,food,"[[62], [452], [439], [204], [173], [493], [153...","[1, 0]"
7,What i enjoy most about palo alto is so many r...,food,"[[267], [173], [487], [33], [225], [497], [198...","[1, 0]"
8,The drinks came out fairly quickly a good two ...,food,"[[153], [245], [93], [388], [154], [449], [157...","[1, 0]"
9,Despite the not so good burger the service was...,food,"[[389], [153], [118], [490], [492], [467], [15...","[1, 0]"


Dynamic RNN은 입력값의 다양한 길이를 고려하여 결과값을 출력하므로 Dynamic RNN이 각 입력값의 실제 길이(단어수)를 할 수 있도록 각 지문별 단어수를 미리 계산해둔다. 

In [11]:
# 지문을 구성하는 단어의 개수를 세는 함수를 만든다.
def word_cnt(paragraph):
    return len(paragraph.split(' '))

# 지문을 구성하는 단어의 개수를 데이터프레임에 추가한다.
df['seq_length'] = df.paragraph.apply(word_cnt)
df

,paragraph,category,enc_paragraph,enc_category,seq_length
0,Dishplace is located in sunnyvale downtown the...,food,"[[240], [111], [294], [77], [435], [253], [530...","[1, 0]",53
1,Srvice can be slower during busy hours but our...,food,"[[346], [132], [5], [314], [135], [262], [19],...","[1, 0]",19
2,Portions are huge both french toast and their ...,food,"[[282], [208], [478], [522], [334], [235], [42...","[1, 0]",42
3,We started with apps going the chicken and waf...,food,"[[68], [266], [257], [364], [428], [153], [128...","[1, 0]",43
4,The biscuits and gravy was too salty two peopl...,food,"[[153], [470], [427], [66], [439], [401], [507...","[1, 0]",82
5,The garlic fries were a great starter (and a h...,food,"[[153], [180], [144], [43], [157], [115], [461...","[1, 0]",24
6,Our meal was excellent i had the pasta ai form...,food,"[[62], [452], [439], [204], [173], [493], [153...","[1, 0]",50
7,What i enjoy most about palo alto is so many r...,food,"[[267], [173], [487], [33], [225], [497], [198...","[1, 0]",43
8,The drinks came out fairly quickly a good two ...,food,"[[153], [245], [93], [388], [154], [449], [157...","[1, 0]",49
9,Despite the not so good burger the service was...,food,"[[389], [153], [118], [490], [492], [467], [15...","[1, 0]",82


텐서플로우의 RNN은 항상 같은 길이의 시퀀스를 받아야 한다.  
길이가 작은 입력 시퀀스는 패딩을 추가적으로 넣어 모든 시퀀스의 길이를 동일하게 설정한다.  
패딩이 RNN 계산에 영향을 끼치지 않도록 Dynamic RNN은 패딩 이전의 입력 시퀀스의 실제 길이를 파라미터로 박아 상태값 계산시 패딩을 제외시킨다.

In [12]:
# 최고로 긴 지문의 단어 수를 구한 후 모든 지문에 패딩을 집어 넣어 최고로 긴 지문과 동일한 길이를 갖도록 만들기 위해 
# 최고로 긴 단어 수를 계산한다.
max_word_cnt = 0
for seq in df.seq_length:
    if max_word_cnt < seq:
        max_word_cnt = seq
# for row in df['paragraph']:
    # if len(row.split(' ')) > max_word_cnt:
        # max_word_cnt = len(row.split(' '))
    
print(max_word_cnt)

91


In [13]:
# 모든 지문에 패딩을 집어 넣어 최고로 긴 지문과 동일한 길이를 갖게하는 함수를 만든다.
def sequence_padding(enc_paragraph):
    seq_length = len(enc_paragraph)
    for i in range(seq_length, max_word_cnt):
        enc_paragraph.append([-1]) # 패딩을 넣는다.
    return enc_paragraph

# 모든 지문에 패딩이 적용된 결과를 데이터프레임에 적용한다.
df['enc_paragraph'] = df.enc_paragraph.apply(sequence_padding)
# 패딩이 적용된 문장을 확인한다.
print(df['enc_paragraph'][11])
print(df['enc_paragraph'][12])

[[153], [101], [439], [54], [141], [87], [393], [58], [280], [157], [384], [77], [46], [427], [488], [310], [160], [31], [79], [145], [342], [312], [143], [503], [381], [530], [365], [211], [395], [184], [157], [466], [324], [157], [109], [448], [157], [258], [414], [11], [77], [400], [237], [126], [284], [130], [358], [40], [427], [465], [112], [77], [149], [473], [28], [388], [184], [25], [536], [26], [289], [257], [260], [381], [265], [37], [349], [492], [386], [166], [165], [528], [283], [284], [77], [157], [514], [280], [160], [95], [0], [79], [153], [17], [77], [216], [427], [441], [-1], [-1], [-1]]
[[207], [496], [160], [113], [75], [434], [185], [77], [3], [529], [79], [306], [160], [528], [514], [87], [393], [276], [420], [421], [337], [165], [224], [321], [244], [267], [157], [95], [367], [479], [165], [111], [383], [153], [2], [129], [22], [315], [56], [514], [398], [195], [90], [153], [168], [354], [153], [334], [239], [75], [217], [339], [20], [148], [409], [165], [105], [

In [14]:
enc_paragraph = np.array(df.enc_paragraph.tolist()) # 입력값
print(type(df.enc_paragraph))
print(type(df.enc_paragraph.tolist()))
print(type(enc_paragraph))
enc_category = np.array(df.enc_category.tolist())   # 실제값
seq_length = np.array(df.seq_length.tolist())       # Dynamic RNN의 2번째 파라미터

train_X = enc_paragraph
print(train_X.shape)
train_Y = enc_category
print(train_Y.shape)

<class 'pandas.core.series.Series'>
<class 'list'>
<class 'numpy.ndarray'>
(20, 91, 1)
(20, 2)


In [16]:
# 이미지

문맥 벡터(contextualized vector) 생성 단계  
인덱스를 임베딩으로 변환한다. 임베딩은 학습 과정을 통해 단어 유사도를 포함하게 되어 문맥 벡터를 생성하는데 도움을 준다.  
인간의 언어(자연어)는 수치화되어 있지 않은 데이터이기 때문에 머신러닝, 딥러닝 기법을 바로 사용할 수 없다. 그래서 자연어 처리에서 특징을 추출해 수치화를 해줘야 하는데 이때 사용하는 것이 '언어의 벡터화' 이다. 이런 벡터화 과정을 Word Embedding이라고 한다.  
LSTM에 임베딩된 시퀀스를 입력해서 최종 상태값을 출력한다. 이 최종 상태값이 문맥 벡터이다.

주제(food, sports) 분류 단계  
문맥 벡터를 덴즈 레이어에 입력하고 출력값을 노드가 2개인 덴즈 레이어에 입력한 후 노드가 2개인 덴즈 레이어의 출력값을 소프트맥스에 입력시켜 food, sports에 대한 예측값(prediction)을 구한다.

위의 그림을 구현한 코드는 아래와 같다.

In [18]:
# 초기화
# tf.reset_default_graph()
tf.set_random_seed(21)

# 학습율과 학습 횟수(epoch)를 설정한다.
learning_rate = 0.001
n_epochs = 300

# 입력값과 출력값을 저장할 placeholder를 만든다.
# 입력값은 단어들의 인덱스이며 그 길이는 항상 91이다.
X = tf.placeholder(tf.float32, [None, max_word_cnt, 1]) # (20, 91, 1) 데이터 갯수, 행, 열
y = tf.placeholder(tf.int32, [None, 2]) # (20, 2) 데이터 갯수, 행

# 워드 임베딩 레이어, 임베딩 레이어는 인덱스를 받아 5차원 벡터의 임베딩을 출력한다.
# tf.layers.dense() 메소드는 densely-connect layer 즉, '완전연결계층'을 만들어준다.
embedding = tf.layers.dense(X, 5)
# LSTM 셀은 64차원 벡터의 생성값을 출력한다.
cell = tf.nn.rnn_cell.LSTMCell(num_units=64)

# 출력값과 생태값을 저장한다. 상태값은 문맥 벡터로 사용되며 dense layer의 입력값이 된다.
output, state = tf.nn.dynamic_rnn(cell, embedding, dtype=tf.float32, sequence_length=seq_length)

# 주제 분류는 두 개의 덴즈 레이어를 사용한다.
# 첫 번째 덴즈 레이어는 32개의 노드를 가지고 있고 두 번째 덴즈 레이어는 2개의 노드를 가지고 있으며 
# 이 2개의 노드가 소프트맥스의 입력값으로 들어간다.
dense_layer = tf.layers.dense(state.h, 32)
# logits는 food, sports를 원 핫 인코딩으로 구분하기 위해 2차원 벡터로 구성한다.
logits = tf.layers.dense(dense_layer, 2)

# 손실 함수 - 크로스 엔트로피
cross_entrophy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)
loss = tf.reduce_mean(cross_entrophy)

# Adam 옵티마이저 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

학습하기

In [20]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1, n_epochs + 1):
        sess.run(optimizer, feed_dict={X: train_X, y: train_Y})
        train_loss = sess.run(loss, feed_dict={X: train_X, y: train_Y})
        
        if epoch == 1 or epoch % 50 == 0:
            preds = tf.nn.softmax(logits)
            correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
            # 정확도 계산
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
            cur_acc = accuracy.eval({X: train_X, y: train_Y})
            print('epoch: {0:3d}, loss: {1:6.4f}, acc: {2:4.2f}'.format(epoch, train_loss, cur_acc))

epoch:   1, loss: 0.8053, acc: 0.50
epoch:  50, loss: 0.4936, acc: 0.80
epoch: 100, loss: 0.1999, acc: 0.95
epoch: 150, loss: 0.1029, acc: 1.00
epoch: 200, loss: 0.0259, acc: 1.00
epoch: 250, loss: 0.0030, acc: 1.00
epoch: 300, loss: 0.0010, acc: 1.00
